In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
dataFile = "/home/jovyan/data400_share/dataset.csv"
columns = pd.read_csv(dataFile, nrows = 1).columns
aggFolder = "Batches/"

In [3]:
columns

Index(['Invoice/Item Number', 'Date', 'Store Number', 'Store Name', 'Address',
       'City', 'Zip Code', 'Store Location', 'County Number', 'County',
       'Category', 'Category Name', 'Vendor Number', 'Vendor Name',
       'Item Number', 'Item Description', 'Pack', 'Bottle Volume (ml)',
       'State Bottle Cost', 'State Bottle Retail', 'Bottles Sold',
       'Sale (Dollars)', 'Volume Sold (Liters)', 'Volume Sold (Gallons)'],
      dtype='object')

In [4]:
def batchAgg(batch, nrows):
    # NOTE: as a way to check that we are chunking correctly, we look at invoice/item number of
    # the columns of the new dataset and the last row of the previous dataset. They should be the
    # same, because the columns of the new dataset will be overwritten, so the first new row of 
    # the new dataset will be the row following the last row of the old dataset
    if batch >= 1:
        data = pd.read_csv(dataFile,
                           nrows = nrows, 
                           skiprows = nrows * batch)
        print("first row:", data.columns.values[0])
        data.columns = columns
    else:
        data = pd.read_csv(dataFile,
                           nrows = nrows)
    agg = data.groupby(['City', 'Date', ])[
    ['Volume Sold (Liters)', 'Bottles Sold', 'Sale (Dollars)']].sum().reset_index()
    agg.to_csv(aggFolder + "agg%d.csv" % batch, index = False)
    val = len(data.index) == nrows
    print("last row:", data.iloc[len(data.index) - 1].values[0])
    return val

In [5]:
# Load data in 1,000,000 rows at a time to aggregate
batch = 0
while batchAgg(batch, 1000000):
    batch += 1

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (6,14) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


last row: INV-03698900072


/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


first row: INV-03698900072
last row: INV-06674400142


/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


first row: INV-06674400142
last row: INV-09367600026
first row: INV-09367600026
last row: INV-12356700060
first row: INV-12356700060
last row: S18978200110
first row: S18978200110
last row: S27349300170
first row: S27349300170
last row: S05165900042
first row: S05165900042
last row: S07065800033
first row: S07065800033
last row: S24466100024
first row: S24466100024
last row: S17738300012
first row: S17738300012
last row: S10166000015
first row: S10166000015
last row: S12846400037
first row: S12846400037
last row: S30473600014
first row: S30473600014
last row: S33318400041
first row: S33318400041
last row: INV-14204700091


In [6]:
aggData = [pd.read_csv(aggFolder + filename) for filename in os.listdir(aggFolder)
          if 'agg' in filename]
aggConcat = pd.concat(aggData)
final = aggConcat.groupby(['City', 'Date', ])[
    ['Volume Sold (Liters)', 'Bottles Sold', 'Sale (Dollars)']].sum().reset_index()

In [7]:
final.to_csv("GrossSales.csv", index=False)

In [8]:
pd.read_csv("GrossSales.csv").head()

,City,Date,Volume Sold (Liters),Bottles Sold,Sale (Dollars)
0,ACKLEY,01/04/2016,114.00,118,1858.58
1,ACKLEY,01/05/2015,163.99,158,1953.04
2,ACKLEY,01/06/2014,146.50,126,1260.57
3,ACKLEY,01/07/2013,98.75,96,955.93
4,ACKLEY,01/08/2013,61.50,72,686.59
